In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense

In [2]:
train = pd.read_csv("../data/raw_data/train/train.csv")

In [3]:
condition = {
    '건고추' : {
        '품종명' : '화건', '거래단위' : '30 kg', '등급' : '상품'
    },
    '사과' : {
        '품종명' : ['홍로', '후지'], '거래단위' : '10 개', '등급' : '상품'
    },
    '감자' : {
        '품종명' : '감자 수미', '거래단위' : '20키로상자', '등급' : '상'
    },
    '배' : {
        '품종명' : '신고', '거래단위' : '10 개', '등급' : '상품'
    },
    '깐마늘(국산)' : {
        '품종명' : '깐마늘(국산)', '거래단위' : '20 kg', '등급' : '상품'
    },
    '무' : {
        '품종명' : '무', '거래단위' : '20키로상자', '등급' : '상'
    },
    '상추' : {
        '품종명' : '청', '거래단위' : '100 g', '등급' : '상품'
    },
    '배추' : {
        '품종명' : '배추', '거래단위' : '10키로망대', '등급' : '상'
    },
    '양파' : {
        '품종명' : '양파', '거래단위' : '1키로', '등급' : '상'
    },
    '대파' : {
        '품종명' : '대파(일반)', '거래단위' : '1키로단', '등급' : '상'
    }
}

In [4]:
# 시계열 데이터를 위한 시퀀스 생성 함수
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length - 3):
        X.append(data[i:i+seq_length, 0])
        y.append(data[i+seq_length:i+seq_length+3, 0])
    return np.array(X), np.array(y)

In [5]:
seq_length = 9  # T-8 ~ T를 사용

In [6]:
sub = pd.read_csv("../data/raw_data/sample_submission.csv")

In [7]:
for i in tqdm(condition):
    try:
        train_c = train[(train['품목명'] == i ) & (train['품종명'] == condition[i]['품종명']) & (train['거래단위'] == condition[i]['거래단위']) & (train['등급'] == condition[i]['등급'])]
    except:
        train_c = train[(train['품목명'] == i ) & (train['품종명'].isin(condition[i]['품종명'])) & (train['거래단위'] == condition[i]['거래단위']) & (train['등급'] == condition[i]['등급'])]
    train_c = train_c[['평균가격(원)']]

    # MinMaxScaler를 이용한 스케일링
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_scaled = scaler.fit_transform(train_c)
    
    X_train, y_train = create_sequences(train_scaled, seq_length)

    # LSTM 모델 생성
    model = Sequential()
    model.add(SimpleRNN(units=50, return_sequences=False, input_shape=(seq_length, 1)))
    model.add(Dense(3))  # T+1, T+2, T+3 예측
    
    model.compile(optimizer='adam', loss='mean_squared_error')

    # 모델 학습
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))  # LSTM에 맞게 데이터 형태 변환
    model.fit(X_train, y_train, epochs=20, batch_size=32)

    k = 0

    for j in range(25):
        test = pd.read_csv(f"../data/raw_data/test/TEST_{j:02d}.csv")
        try:
            test = test[(test['품목명'] == i ) & (test['품종명'] == condition[i]['품종명']) & (test['거래단위'] == condition[i]['거래단위']) & (test['등급'] == condition[i]['등급'])]
        except:
            test = test[(test['품목명'] == i ) & (test['품종명'].isin(condition[i]['품종명'])) & (test['거래단위'] == condition[i]['거래단위']) & (test['등급'] == condition[i]['등급'])]
        
        test = test[['평균가격(원)']]
        test_scaled = scaler.fit_transform(test)

        # 테스트 데이터도 시퀀스 형태로 변환
        X_test = np.array([test_scaled.flatten()])  # 테스트 데이터의 마지막 9개 시점을 입력으로 사용
        X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))
        
        # 예측 수행
        predicted = model.predict(X_test)
        
        # 예측값을 원래 스케일로 역변환
        predicted_prices = scaler.inverse_transform(np.concatenate((predicted, np.zeros((predicted.shape[0], 1))), axis=1))[:, :3]
        
        # T+1, T+2, T+3 값을 개별적으로 출력
        T_plus_1 = predicted_prices[0, 0]
        T_plus_2 = predicted_prices[0, 1]
        T_plus_3 = predicted_prices[0, 2]

        sub.loc[k, i] = T_plus_1
        k += 1
        sub.loc[k, i] = T_plus_2
        k += 1
        sub.loc[k, i] = T_plus_3
        k += 1

  0%|                                                                                           | 0/10 [00:00<?, ?it/s]C:\Users\insu\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.2848
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0654 
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0466 
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0424 
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0274 
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0283 
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0272 
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0196 
Epoch 9/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0162 
Epoch 10/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0184 
Epoch 11/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0161 
Epoch 12/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0128 
Epoch 13/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0124 
Epoch 14/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0146 
Epoch 15/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0130 
Epoch 16/20
5/5 ━━━━━━━━━━━━━━━━━━━

 10%|████████▎                                                                          | 1/10 [00:04<00:40,  4.46s/it]

Epoch 1/20


C:\Users\insu\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [00:33<00:03,  3.61s/it]

Epoch 1/20


C:\Users\insu\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 0.0752  
Epoch 2/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0397 
Epoch 3/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0334 
Epoch 4/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0294 
Epoch 5/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0214 
Epoch 6/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0236 
Epoch 7/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0182 
Epoch 8/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0152 
Epoch 9/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0158 
Epoch 10/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0167 
Epoch 11/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0159 
Epoch 12/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0138 
Epoch 13/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0134 
Epoch 14/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0122 
Epoch 15/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0130 
Epoch 16/20
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:36<00:00,  3.67s/it]


In [8]:
sub.to_csv("../data/submission/sub_02.csv", index=False)